<a href="https://colab.research.google.com/github/pranjal4697/GAN-101/blob/master/GAN_Simple_Cubic_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#import all the libraries here
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential

In [0]:
def generate_real_samples(number_of_samples):
    #generate 'number_of_samples' samples between -0.5 and 0.5
    X1 = np.random.rand(number_of_samples) - 0.5
    X2 = X1 * X1 * X1
    X1 = X1.reshape(number_of_samples, 1)
    X2 = X2.reshape(number_of_samples, 1)
    x_real = np.hstack((X1, X2))
    y_real = np.ones((number_of_samples, 1))
    return x_real, y_real

def generate_random_vector(number, vector_length=5):
    x = np.random.randn(number*vector_length)
    x = x.reshape((number, vector_length))
    return x

def generate_fake_samples(generator, number_of_samples, latent_dim=5):
    #generate 'number_of_samples' fake samples
    X1 = generate_random_vector(number=number_of_samples, vector_length=latent_dim)
    x_fake = generator.predict(X1)
    y_fake = np.zeros((number_of_samples, 1))
    return x_fake, y_fake

In [0]:
def build_generative(input_dim=5):
    model = Sequential()
    model.add(Dense(15, activation='relu', input_dim=input_dim))
    model.add(Dense(2, activation='linear'))
    return model

def build_discriminative(input_dim=2):
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=input_dim))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_gan(gmodel, dmodel):
    dmodel.trainable = False
    model = Sequential()
    model.add(gmodel)
    model.add(dmodel)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    # prepare real samples
    x_real, y_real = generate_real_samples(n)
    # evaluate discriminator on real examples
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(generator, n, latent_dim)
    # evaluate discriminator on fake examples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print(epoch, acc_real, acc_fake)
    # scatter plot real and fake data points
    plt.scatter(x_real[:, 0], x_real[:, 1], color='red')
    plt.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
    # save plot to file
    filename = '/content/drive/My Drive/generated_plot_e%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()


In [0]:
def train(g_model, d_model, gan_model, epochs=10000, batch_size=128, latent_dim=5, n_eval=2000):
    half_batch = int(batch_size/2)
    for i in range(epochs):
        x_real, y_real = generate_real_samples(half_batch)
        x_fake, y_fake = generate_fake_samples(generator=g_model, number_of_samples=half_batch, latent_dim=latent_dim)
        d_model.train_on_batch(x_real, y_real)
        d_model.train_on_batch(x_fake, y_fake)
        x_gan = generate_random_vector(batch_size, latent_dim)
        y_gan = np.ones((batch_size, ))
        gan_model.train_on_batch(x_gan, y_gan)
        if (i+1) % n_eval == 0:
            summarize_performance(i, g_model, d_model, latent_dim)


In [114]:
latent_dim = 5
# create the discriminator
discriminator = build_discriminative()
# create the generator
generator = build_generative(latent_dim)
# create the gan
gan_model = build_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1999 0.30000001192092896 1.0
3999 0.3499999940395355 0.30000001192092896
5999 0.5199999809265137 0.6000000238418579
7999 0.47999998927116394 0.5299999713897705
9999 0.4399999976158142 0.5400000214576721
